# Iterative LQG

This notebook implements the ILQG method, as described in the following paper:

[A generalized iterative LQG method for locally-optimal feedback
control of constrained nonlinear stochastic systems](http://maeresearch.ucsd.edu/skelton/publications/weiwei_ilqg_CDC43.pdf)

In general, suppose that we have the following dynamical system:

$$ x_{k+1} = f(x_{k}, u_{k})$$
$$ J_0 = \frac{1}{2} (x_N - x^*)^TQ_f(x_N - x^*) + \frac{1}{2} \sum_{k=0}^{T-1}(x_k^TQx_k + u_k^TRu_k) $$

In [89]:
import numpy as np

# The problem that iLQG faces is kind of a different flavor
# from typical RL problems. Here we usually assume a nonlinear dynamical system,
# whose cost can be locally approximated by a positive definite quadratic cost,
# and where the dynamics can be locally linearized.
# Let's first port the matlab version of iLQR to Python. The matlab version was authored
# by Timothy Lillicrap (http://limb.biomed.queensu.ca/lab_members/timothylillicrap/projects.php)

def pendulum_dynamics(x, u, dt):
    g = 9.8
    m = 1.0 
    l = 1.0 # length of the rod
    mu = 0.01 # friction coefficient
    th, thdot = x
    u0 = u[0]
    newth = th + thdot*dt
    newthdot = thdot + ( (-mu/(m*l**2))*thdot + (g/l)*np.sin(th) + u/(m*l**2) ) * dt
    return np.array([newth, newthdot])

print pendulum_dynamics(np.array([np.pi, 0]), np.array([0]), 0.005)

[  3.14159265e+00   6.00076932e-18]


In [90]:
def ilqr_forward_pass(x0, uref, sysdyn, cost_func, final_cost_func, K=None, xref=None):
    
    Dx = len(x0)
    N, Du = uref.shape
    x = np.zeros((N+1, Dx))
    uout = np.zeros((N, Du))
    cost = 0
    x[0] = x0
    for t in range(N):
        if K is not None and xref is not None:
            u = uref[t] + K[t].dot(x[t] - xref[t])
        else:
            u = uref[t]
        x[t+1] = sysdyn(x[t], u)
        cost += cost_func(x[t], u)
        uout[t] = u
    cost += final_cost_func(x[N])
    return x, cost, uout

In [91]:
def fd_jacobian(x, f, eps=1e-5):
    Nx = len(x)
    Nf = len(f(x))
    J = np.zeros((Nf, Nx))
    eyex = np.eye(Nx)
    for dx in range(Nx):
        xp = x + eyex[dx] * eps
        xn = x - eyex[dx] * eps
        J[:, dx] = (f(xp) - f(xn)) / (2*eps)
    return J

def fd_grad(x, f, eps=1e-5):
    Nx = len(x)
    g = np.zeros(Nx)
    eyex = np.eye(Nx)
    for dx in range(Nx):
        xp = x + eyex[dx] * eps
        xn = x - eyex[dx] * eps
        g[dx] = (f(xp) - f(xn)) / (2*eps)
    return g

def ilqr_linearize(x, u, sysdyn, cost_func, final_cost_func):
    
    Dx = x.shape[1]
    N, Du = u.shape
    fx = np.zeros((N, Dx, Dx))
    fu = np.zeros((N, Dx, Du))
    cx = np.zeros((N+1, Dx))
    cu = np.zeros((N, Du))
    cxx = np.zeros((N+1, Dx, Dx))
    cuu = np.zeros((N, Du, Du))
    cxu = np.zeros((N, Dx, Du))

    for k in range(N):
        fx[k] = jacobian(x[k], lambda x: sysdyn(x, u[k]))
        fu[k] = jacobian(u[k], lambda u: sysdyn(x[k], u))
        cx[k] = grad(x[k], lambda x: cost_func(x, u[k]))
        cu[k] = grad(u[k], lambda u: cost_func(x[k], u))
        cxu[k] = jacobian(u[k], lambda u: grad(x[k], lambda x: cost_func(x, u)))
        cuu[k] = jacobian(u[k], lambda u: grad(u, lambda u: cost_func(x[k], u)))
        
    cx[N] = grad(x[N], final_cost_func)
    cxx[N] = jacobian(x[N], lambda x: grad(x, final_cost_func))
    return fx, fu, cx, cu, cxx, cxu, cuu

In [92]:
def ilqr_backward_pass(x, u, sysdyn, cost_func, final_cost_func):
    Dx = x.shape[1]
    N, Du = u.shape
    fx, fu, cx, cu, cxx, cxu, cuu = ilqr_linearize(x, u, sysdyn, cost_func, final_cost_func)
    
    Vx = np.zeros((N+1, Dx))
    Vxx = np.zeros((N+1, Dx, Dx))
    Vx[N] = cx[N]
    Vxx[N] = cxx[N]
    k = np.zeros((N, Du))
    K = np.zeros((N, Du, Dx))
    
    # dVx and dVxx are used for computing the expected reduction in cost
    # dV ~ -alpha*Q_u.T*k - alpha^2*k.T*Q_uu*k
    dVx = 0
    dVxx = 0
    
    for t in range(N-1, -1, -1):
        Qx = cx[t] + fx[t].T.dot(Vx[t+1])
        Qu = cu[t] + fu[t].T.dot(Vx[t+1])
        Qxx = cxx[t] + fx[t].T.dot(Vxx[t+1]).dot(fx[t])
        # TODO regularize
        Qux = cxu[t].T + fu[t].T.dot(Vxx[t+1]).dot(fx[t])
        Quu = cuu[t] + fu[t].T.dot(Vxx[t+1]).dot(fu[t])
        
        # TODO regularize
        k[t] = -np.linalg.inv(Quu)*Qu
        K[t] = -np.linalg.inv(Quu)*Qux
        
        Vx[t] = Qx - K[t].T.dot(Quu).dot(k[t])
        Vxx[t] = Qxx - K[t].T.dot(Quu).dot(K[t])
        
        dVx += np.inner(k[t], Qu)
        dVxx += 0.5*k[t].T.dot(Quu).dot(k[t])
    return Vx, Vxx, k, K, dVx, dVxx

In [93]:
def ilqr(x0, u0, sysdyn, cost_func, final_cost_func,
         max_iter=20,
         min_reduction=0,
         max_line_search_iter=8,
        ):
    
    Dx = len(x0)
    Du = u0.shape[1]
    N = len(u0)
    
    u = u0
    
    for itr in range(max_iter):
        x, cost, _ = ilqr_forward_pass(x0, u, sysdyn, cost_func, final_cost_func)
        print cost
        Vx, Vxx, k, K, dVx, dVxx = ilqr_backward_pass(x, u, sysdyn, cost_func, final_cost_func)
        alpha = 1
        for _ in range(max_line_search_iter):
            # unew is different from u+alpha*k because of the feedback control term
            xnew, cnew, unew = ilqr_forward_pass(x0, u+alpha*k, sysdyn, cost_func, final_cost_func, K, x)
            dcost = cost - cnew
            expected = -alpha*(dVx+alpha*dVxx)
            if expected < 0:
                raise ValueError('should not happen')
            if dcost / expected > min_reduction:
                break
            else:
                alpha = alpha * 0.5
        u = unew
        if abs(cost - cnew) < 1e-6:
            print 'converged'
            break
        cost = cnew

u0 = np.ones((120, 1))*1000
x0 = np.array([np.pi, 0])
xf = np.array([0, 0])
Qf = np.diag([1, 1])
Q = np.zeros((2, 2))
R = 1e-5*np.eye(1)
dt = 5e-3
n_itr = 20
threshold = 0
    
ilqr(x0, u0, (lambda x, u: pendulum_dynamics(x, u, dt)), (lambda x, u: 0.5*x.T.dot(Q).dot(x) + 0.5*u.T.dot(R).dot(u)), (lambda x: 0.5*(x - xf).T.dot(Qf).dot(x - xf)))

195787.655077
37.182959596
0.696063905125
0.468757617223
0.468726219721
converged
